# Import Necessary Libraries


In [140]:
import re
import qalsadi.lemmatizer as lemmatizer
import nltk
import pymongo
from nltk.tokenize import word_tokenize, sent_tokenize, TreebankWordTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.isri import ISRIStemmer

# Connect with mongoDB to fetchDATA from news_data

In [141]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["news_data"]
collection = db["addustourspider"]

documents = collection.find()

# Function contain techniques to clean text and Normalisation and Discretization

In [142]:
def clean_text(text):
    # Remove HTML Tags
    html = re.compile(r'<[^>]+>')
    text = html.sub('', text)

    # Remove URLs
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub('', text)

    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)

    # Remove Email IDs
    email = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    text = email.sub('', text)

    # Remove Arabic diacritics (vowels and accents)
    arabic_diacritics = re.compile(r'[\u064B-\u065F\u0640]')
    text = arabic_diacritics.sub('', text)

    return text


# Lists to store original and cleaned texts

In [143]:
original_texts = []
cleaned_texts = []

# Loop through documents and clean text

In [144]:
for document in documents:
    original_text = document['content']
    cleaned_text = clean_text(original_text)

    # Adjust the limit in the 5 because will be often happens an error when trying to display large amounts of data after this boucle
    if len(original_texts) < 1:
        original_texts.append(original_text)
        cleaned_texts.append(cleaned_text)
    else:
        break
client.close()

# Tokenization

In [145]:
tokenizer = TreebankWordTokenizer()
Puncter = WordPunctTokenizer()
Regex = RegexpTokenizer("\w+|[^\w\s]")

# Tokenize cleaned text with all types to see the difference between them

In [146]:
for  cleaned_ in cleaned_texts:
    #print("\nOriginal Text:\n")
    #print(original_)
    #print("\nCleaned Text:\n")
    #print(cleaned_)
    print('\n the result with Sentence Tokenization – Splitting sentences in the paragraph: \n')
    print(sent_tokenize(cleaned_))
    print("\n------------------------------\n")
    print('\n the result with Word Tokenization – Splitting words in a sentence: \n')
    print(word_tokenize(cleaned_))
    print('\n the result with Word Tokenization – Using TreebankWordTokenizer: \n') #These tokenizers work by separating the words using punctuation and spaces.
    print(tokenizer.tokenize(cleaned_))
    print('\n the result with Word Tokenization – Using PunktWordTokenizer: \n')
    words = Puncter.tokenize(cleaned_)
    print(words)
    print('\n the result with Word Tokenization – Using Regular Expression: \n')
    print(Regex.tokenize(cleaned_))


 the result with Sentence Tokenization – Splitting sentences in the paragraph: 

['شعرت بخسارة مريرة لأنني لم أكن بين جمهور مباراة كرة القدم الجميلة التي جمعت قبل أيام الفريقين العريقين الفيصلي والوحدات كم كنت سأكسب لو أني كنت معهم صوتا صادحا واحدا موحدا في هتاف بليغ قوي أغاظ ويغيظ أبناء الماء العكر وصياديه وينحاز إلى أردن عظيم بأطيافه وألوانه كوطن نحبه ونفديه بالمهج ولم يكن يوما إلا مع أهله في غزة وفلسطين وسندا لكل جرح عربي الهتافات العظيمة لامست شغاف قلبي ورقصتني بهجة فرغم أن أصوات غرابية عديدة كانت تبث سوداويتها وشؤمها مطالبة بإلغاء المباراة في الظرف العصيب الذي نمر فيه لأنهم يتخوفون من انقلاب المباراة إلى معركة دق خشوم و كسر عظم وطحن رؤوس لكني لم أكن أخاف أبدا من تلك المواجهة لأنني واثق في متانة وقوة مسبحتنا الوطنية\xa0 \xa0 لدي مسبحة مطعمة بالفضة عمرها ثلاثون عاما لها ذكريات طيبة في نفسي لكني لا أحبذ أن أظهرها ليس لأني أخشى عليها الانفراط حتى حين أدبك بها لا سمح الله بل لأنك قد تقف مع أحدهم فيمد يده لا شعوريا ويأخذها يتأملها ويداعب شراشيبها ويتفحص حجارتها ويسألك إن كانت حقا فيروز

# Stop words

In [147]:
stop_words = set(stopwords.words('arabic'))

# Filter words after applying stop words techniques

In [148]:
print('\n  Filtred words after applying the Stop words techniques: \n ')
Filtered_text = [word for word in words if not word in stop_words]
#Filtered_tafter ext = ' '.join(Filtered_text)
print(Filtered_text)


  Filtred words after applying the Stop words techniques: 
 
['شعرت', 'بخسارة', 'مريرة', 'لأنني', 'أكن', 'جمهور', 'مباراة', 'كرة', 'القدم', 'الجميلة', 'جمعت', 'أيام', 'الفريقين', 'العريقين', 'الفيصلي', 'والوحدات', 'كنت', 'سأكسب', 'أني', 'كنت', 'معهم', 'صوتا', 'صادحا', 'واحدا', 'موحدا', 'هتاف', 'بليغ', 'قوي', 'أغاظ', 'ويغيظ', 'أبناء', 'الماء', 'العكر', 'وصياديه', 'وينحاز', 'أردن', 'عظيم', 'بأطيافه', 'وألوانه', 'كوطن', 'نحبه', 'ونفديه', 'بالمهج', 'ولم', 'يكن', 'يوما', 'أهله', 'غزة', 'وفلسطين', 'وسندا', 'لكل', 'جرح', 'عربي', 'الهتافات', 'العظيمة', 'لامست', 'شغاف', 'قلبي', 'ورقصتني', 'بهجة', 'فرغم', 'أصوات', 'غرابية', 'عديدة', 'كانت', 'تبث', 'سوداويتها', 'وشؤمها', 'مطالبة', 'بإلغاء', 'المباراة', 'الظرف', 'العصيب', 'نمر', 'لأنهم', 'يتخوفون', 'انقلاب', 'المباراة', 'معركة', 'دق', 'خشوم', 'كسر', 'عظم', 'وطحن', 'رؤوس', 'لكني', 'أكن', 'أخاف', 'المواجهة', 'لأنني', 'واثق', 'متانة', 'وقوة', 'مسبحتنا', 'الوطنية', 'لدي', 'مسبحة', 'مطعمة', 'بالفضة', 'عمرها', 'عاما', 'ذكريات', 'طيبة', 'نفسي', 'لكني', 

# Stemming and Lemmatization

In [149]:
lemmatizer = lemmatizer.Lemmatizer()
ISRI = ISRIStemmer()

In [150]:
print("\n-------------Stemming and Lemmatization Part----------------\n")
stemmed_words = [ISRI.stem(word) for word in Filtered_text]
print("Stemmed Words:")
print(stemmed_words)


-------------Stemming and Lemmatization Part----------------

Stemmed Words:
['شعر', 'خسر', 'رير', 'لأن', 'اكن', 'جمهور', 'برا', 'كرة', 'قدم', 'جمل', 'جمع', 'ايم', 'فرق', 'عرق', 'يصل', 'وحد', 'كنت', 'أكسب', 'اني', 'كنت', 'عهم', 'صوت', 'صدح', 'وحد', 'وحد', 'هتف', 'بلغ', 'قوي', 'اغظ', 'يغظ', 'بنء', 'ماء', 'عكر', 'صيد', 'نحز', 'ارد', 'عظم', 'أطياف', 'ألو', 'كوط', 'نحب', 'نفد', 'مهج', 'ولم', 'يكن', 'يوم', 'اهل', 'غزة', 'لسط', 'سند', 'لكل', 'جرح', 'عرب', 'هتف', 'عظم', 'لمس', 'شغف', 'قلب', 'رقص', 'بهج', 'رغم', 'اصو', 'غرب', 'عدد', 'كانت', 'تبث', 'ودو', 'شؤم', 'طلب', 'إلغاء', 'برا', 'ظرف', 'عصب', 'نمر', 'لأن', 'تخف', 'قلب', 'برا', 'عرك', 'دق', 'خشم', 'كسر', 'عظم', 'وطح', 'رؤس', 'لكن', 'اكن', 'اخف', 'وجه', 'لأن', 'وثق', 'تان', 'وقة', 'سبح', 'وطن', 'لدي', 'سبح', 'طعم', 'فضة', 'عمر', 'عما', 'ذكر', 'طيب', 'نفس', 'لكن', 'حبذ', 'ظهر', 'لأن', 'خشى', 'عليها', 'فرط', 'ادب', 'سمح', 'الل', 'لأن', 'تقف', 'احد', 'يمد', 'يده', 'شعر', 'أخذ', 'أمل', 'دعب', 'شراشيب', 'فحص', 'حجر', 'سأل', 'كانت', 'يرز', 'تهز'

In [151]:
lemmatized_word = [lemmatizer.lemmatize(word) for word in Filtered_text]
print("\nLemmatized Words:")
print(lemmatized_word)


Lemmatized Words:
['شعر', 'خسار', 'مرير', 'لأنني', 'كنى', 'جمهور', 'مباراة', 'كرة', 'قدم', 'جميل', 'جمع', 'أيام', 'فريق', 'عريق', 'فيصل', 'وحد', 'كنت', 'كسب', 'أنة', 'كنت', 'مع', 'صوت', 'صادح', 'واحد', 'موحد', 'هتاف', 'بليغ', 'قوة', 'أغاظ', 'أغاظ', 'بناء', 'ماء', 'عكر', 'صياد', 'انحاز', 'أردن', 'عظيم', 'أطياف', 'ألوان', 'وطن', 'نحب', 'فدى', 'مهج', 'لم', 'كنى', 'يوم', 'هل', 'غزة', 'فلسطين', 'سند', 'كل', 'جرح', 'عرب', 'هتاف', 'عظيم', 'لامس', 'شغاف', 'قلب', 'رقص', 'بهج', 'رغم', 'أصوات', 'غراب', 'عديد', 'كانت', 'باث', 'سوداوي', 'شؤم', 'مطالب', 'إلغاء', 'مباراة', 'ظرف', 'عصيب', 'نمر', 'أن', 'تخوف', 'انقلاب', 'مباراة', 'معركة', 'دق', 'خشوم', 'كسر', 'عظم', 'طحن', 'رؤوس', 'كنة', 'كنى', 'أخافي', 'مواجه', 'لأنني', 'واثق', 'متان', 'قوة', 'مسبح', 'وطن', 'لد', 'مسبح', 'مطعم', 'فض', 'عمر', 'عام', 'ذكر', 'طيب', 'نفس', 'كنة', 'حبذ', 'ظهر', 'أنة', 'خشى', 'على', 'انفراط', 'أدب', 'سمح', 'الله', 'أن', 'تقفي', 'أحد', 'أمد', 'يد', 'شعور', 'أخذ', 'تأمل', 'داعب', 'شراشيبها', 'تفحص', 'حجار', 'سأل', 'كانت', 'ف

#### Comparing IRISTEMMER and Qalsadi Lemmatizer Results:

Here's a comparison of the stemming and lemmatization results you provided:

Similarities:

    Both techniques aim to reduce words to their base or root form.

    Both methods successfully handled most of the words, reducing them to recognizable roots.

    Both outputs maintain the general flow and meaning of the original text.

Differences:

    Output Form:

        IRISTEMMER: Produces stems, which may not always be actual words but are linguistic roots.

        Qalsadi: Produces lemmas, which are actual words in the language.

    Word Accuracy:

        IRISTEMMER: May produce less accurate representations, especially for irregular words.

        Qalsadi: Generally provides more accurate lemmas due to its dictionary-based approach and understanding of Arabic morphology.

    Word Length:

        IRISTEMMER: Often creates shorter forms as it focuses on stripping suffixes.

        Qalsadi: May retain some suffixes or prefixes depending on the word and its grammatical role.

Examples:

    "شعرت" (felt):

        IRISTEMMER: "شعر" (hair, poetry) - Less accurate, loses the meaning of feeling.

        Qalsadi: "شعر" (felt) - Maintains the correct meaning and is a valid word.

    "جمهور" (audience):

        IRISTEMMER: "جمهور" (republic) - Stem is accurate but has a different meaning.

        Qalsadi: "جمهور" (audience) - Correct lemma with the intended meaning. 

# Perform Part-Of-Speech tagging

### Perform Parts of Speech technics based on Rule based approach

In [152]:
import spacy
nlp = spacy.load("en_core_web_sm")

#Filtered data to a single string
data  = ' '.join(Filtered_text)

#process with SpaCy nlp
content = nlp(data)

for word in content:
    #word     #word with process of pos  #word with process of pos+explanation
    print(word , "//",word.pos_  , "//" ,spacy.explain(word.pos_))

شعرت // PROPN // proper noun
بخسارة // PROPN // proper noun
مريرة // PROPN // proper noun
لأنني // PROPN // proper noun
أكن // PROPN // proper noun
جمهور // PROPN // proper noun
مباراة // PROPN // proper noun
كرة // PROPN // proper noun
القدم // PROPN // proper noun
الجميلة // PROPN // proper noun
جمعت // PROPN // proper noun
أيام // PROPN // proper noun
الفريقين // PROPN // proper noun
العريقين // PROPN // proper noun
الفيصلي // PROPN // proper noun
والوحدات // PROPN // proper noun
كنت // PROPN // proper noun
سأكسب // PROPN // proper noun
أني // PROPN // proper noun
كنت // PROPN // proper noun
معهم // PROPN // proper noun
صوتا // NOUN // noun
صادحا // PROPN // proper noun
واحدا // PROPN // proper noun
موحدا // PROPN // proper noun
هتاف // PROPN // proper noun
بليغ // PROPN // proper noun
قوي // PROPN // proper noun
أغاظ // PROPN // proper noun
ويغيظ // PROPN // proper noun
أبناء // PROPN // proper noun
الماء // PROPN // proper noun
العكر // PROPN // proper noun
وصياديه // PROPN // pro

## So as we see in the result is not good and not bad for SpaCy POS tagger 

### Perform Parts of Speech technics based Machine learning approaches with model arabic.tagger

In [153]:
from nltk.tag import StanfordPOSTagger

# Set the absolute paths to the Stanford POS Tagger jar file and model file
jar_path = '/home/hafdaoui/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/stanford-postagger.jar'
model_path = '/home/hafdaoui/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/models/arabic.tagger'

tagger = StanfordPOSTagger(model_path, jar_path)

# Tag the tokens
tagged_tokens = tagger.tag(Filtered_text)

# Print the tagged tokens
print(tagged_tokens)


[('', 'شعرت/VBD'), ('', 'بخسارة/NN'), ('', 'مريرة/JJ'), ('', 'لأنني/JJ'), ('', 'أكن/VBD'), ('', 'جمهور/NN'), ('', 'مباراة/NN'), ('', 'كرة/NN'), ('', 'القدم/DTNN'), ('', 'الجميلة/DTJJ'), ('', 'جمعت/VBD'), ('', 'أيام/NN'), ('', 'الفريقين/DTNNS'), ('', 'العريقين/DTJJ'), ('', 'الفيصلي/DTNNP'), ('', 'والوحدات/NNS'), ('', 'كنت/VBD'), ('', 'سأكسب/NN'), ('', 'أني/JJ'), ('', 'كنت/VBD'), ('', 'معهم/NN'), ('', 'صوتا/NN'), ('', 'صادحا/JJ'), ('', 'واحدا/JJ'), ('', 'موحدا/JJ'), ('', 'هتاف/NN'), ('', 'بليغ/NN'), ('', 'قوي/JJ'), ('', 'أغاظ/VBD'), ('', 'ويغيظ/NNP'), ('', 'أبناء/VBG'), ('', 'الماء/DTNN'), ('', 'العكر/DTNN'), ('', 'وصياديه/NNP'), ('', 'وينحاز/NNP'), ('', 'أردن/NN'), ('', 'عظيم/JJ'), ('', 'بأطيافه/NN'), ('', 'وألوانه/NNP'), ('', 'كوطن/NNP'), ('', 'نحبه/VBP'), ('', 'ونفديه/NNP'), ('', 'بالمهج/NNP'), ('', 'ولم/NNP'), ('', 'يكن/VBP'), ('', 'يوما/NN'), ('', 'أهله/NN'), ('', 'غزة/NNP'), ('', 'وفلسطين/NNP'), ('', 'وسندا/NNP'), ('', 'لكل/VBD'), ('', 'جرح/NN'), ('', 'عربي/JJ'), ('', 'الهتافات/DTN

## but the  Stanford Arabic POS Tagger work so good and we get wonderful result

# Comparing SpaCy Rule-Based and Stanford Tagger Machine Learning POS Tagging:

Here's a comparison of the POS tagging results you obtained using SpaCy's rule-based approach and the Stanford Tagger's machine learning model:

General Observations:

    SpaCy with en_core_web_sm:

        Incorrectly identifies most words as proper nouns (PROPN). This suggests a mismatch between the language model and your Arabic text.

        Limited accuracy due to the model's inability to handle Arabic morphology and context.

    Stanford Tagger with Arabic Model:

        Demonstrates much better performance, correctly identifying various parts of speech including nouns (NN), verbs (VBD, VBP), adjectives (JJ), and more.

        Leverages machine learning for better accuracy and understanding of Arabic grammatical structure.

# NAMED ENTITE RECOGNITION

In [154]:
import stanza
nlp = stanza.Pipeline('ar', processors='tokenize,ner') # initialize English neural pipeline
doc = nlp(cleaned_) # run annotation over a sentence
for entity in doc.ents:
    print(f"Entity: {entity.text}\t Type: {entity.type}")

2024-04-08 02:05:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 02:05:22 INFO: Downloaded file to /home/hafdaoui/stanza_resources/resources.json
2024-04-08 02:05:22 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 02:05:22 INFO: Loading these models for language: ar (Arabic):
| Processor | Package      |
----------------------------
| tokenize  | padt         |
| mwt       | padt         |
| ner       | aqmar_charlm |

2024-04-08 02:05:22 INFO: Using device: cpu
2024-04-08 02:05:22 INFO: Loading: tokenize
2024-04-08 02:05:22 INFO: Loading: mwt
2024-04-08 02:05:22 INFO: Loading: ner
2024-04-08 02:05:23 INFO: Done loading processors!


Entity: غزة	 Type: LOC
Entity: وفلسطين	 Type: LOC
Entity: بغداد	 Type: LOC


In [155]:
print(doc)
print(doc.entities)

[
  [
    {
      "id": 1,
      "text": "شعرت",
      "start_char": 0,
      "end_char": 4,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": [
        2,
        3
      ],
      "text": "بخسارة",
      "start_char": 5,
      "end_char": 11,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "ب"
    },
    {
      "id": 3,
      "text": "خسارة"
    },
    {
      "id": 4,
      "text": "مريرة",
      "start_char": 12,
      "end_char": 17,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": [
        5,
        6
      ],
      "text": "لأنني",
      "start_char": 18,
      "end_char": 23,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 5,
      "text": "لأن"
    },
    {
      "id": 6,
      "text": "ني"
    },
    {
      "id": 7,
      "text": "لم",
      "start_char": 24,
      "end_char": 26,
      "ner": "O",
      "multi_ner": 